### Student Information
Name:

Student ID:

GitHub ID:

---

### Instructions

1. First: do the **take home** exercises in the [DM2024-Lab1-Master](https://github.com/didiersalazar/DM2024-Lab1-Master.git). You may need to copy some cells from the Lab notebook to this notebook. __This part is worth 20% of your grade.__


2. Second: follow the same process from the [DM2024-Lab1-Master](https://github.com/didiersalazar/DM2024-Lab1-Master.git) on **the new dataset**. You don't need to explain all details as we did (some **minimal comments** explaining your code are useful though).  __This part is worth 30% of your grade.__
    - Download the [the new dataset](https://huggingface.co/datasets/Senem/Nostalgic_Sentiment_Analysis_of_YouTube_Comments_Data). The dataset contains a `sentiment` and `comment` columns, with the sentiment labels being: 'nostalgia' and 'not nostalgia'. Read the specificiations of the dataset for background details. 
    - You are allowed to use and modify the `helper` functions in the folder of the first lab session (notice they may need modification) or create your own.


3. Third: please attempt the following tasks on **the new dataset**. __This part is worth 30% of your grade.__
    - Generate meaningful **new data visualizations**. Refer to online resources and the Data Mining textbook for inspiration and ideas. 
    - Generate **TF-IDF features** from the tokens of each text. This will generating a document matrix, however, the weights will be computed differently (using the TF-IDF value of each word per document as opposed to the word frequency). Refer to this Scikit-learn [guide](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) .
    - Implement a simple **Naive Bayes classifier** that automatically classifies the records into their categories. Use both the TF-IDF features and word frequency features to build two seperate classifiers. Note that for the TF-IDF features you might need to use other type of NB classifier different than the one in the Master Notebook. Comment on the differences.  Refer to this [article](https://hub.packtpub.com/implementing-3-naive-bayes-classifiers-in-scikit-learn/).


4. Fourth: In the lab, we applied each step really quickly just to illustrate how to work with your dataset. There are somethings that are not ideal or the most efficient/meaningful. Each dataset can be handled differently as well. What are those inefficent parts you noticed? How can you improve the Data preprocessing for these specific datasets? __This part is worth 10% of your grade.__


5. Fifth: It's hard for us to follow if your code is messy, so please **tidy up your notebook** and **add minimal comments where needed**. __This part is worth 10% of your grade.__


You can submit your homework following these guidelines: [Git Intro & How to hand your homework](https://github.com/didiersalazar/DM2024-Lab1-Master/blob/main/Git%20Intro%20%26%20How%20to%20hand%20your%20homework.ipynb). Make sure to commit and save your changes to your repository __BEFORE the deadline (October 27th 11:59 pm, Sunday)__. 

In [1]:
### Begin Assignment Here
from datasets import load_dataset

ds = load_dataset("Senem/Nostalgic_Sentiment_Analysis_of_YouTube_Comments_Data")

ModuleNotFoundError: No module named 'datasets'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer

dataset = load_dataset("Senem/Nostalgic_Sentiment_Analysis_of_YouTube_Comments_Data")
df = pd.DataFrame(dataset['train'])

nostalgia_comments = df[df['sentiment'] == 'nostalgia']['comment']
not_nostalgia_comments = df[df['sentiment'] == 'not nostalgia']['comment']

# CountVectorizer詞頻
vectorizer = CountVectorizer(stop_words='english', max_features=10)  # max_features=10 表示取出最常見的10個詞
nostalgia_word_counts = vectorizer.fit_transform(nostalgia_comments).toarray().sum(axis=0)
not_nostalgia_word_counts = vectorizer.fit_transform(not_nostalgia_comments).toarray().sum(axis=0)

words = vectorizer.get_feature_names_out()
nostalgia_df = pd.DataFrame({'word': words, 'count': nostalgia_word_counts, 'label': 'nostalgia'})
not_nostalgia_df = pd.DataFrame({'word': words, 'count': not_nostalgia_word_counts, 'label': 'not nostalgia'})
word_counts_df = pd.concat([nostalgia_df, not_nostalgia_df])

# 畫出長條圖
plt.figure(figsize=(12, 6))
sns.barplot(data=word_counts_df, x='word', y='count', hue='label')
plt.title('Nostalgia & Not Nostalgia')
plt.legend(title='lable')
plt.xticks(rotation=45)
plt.show()
sns.countplot(data=df, x='sentiment')
plt.show()

# 詞雲
all_text = ' '.join(df['comment'])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

# 計算情感標籤的比例
sentiment_counts = df['sentiment'].value_counts()
labels = sentiment_counts.index
sizes = sentiment_counts.values

# 繪製圓餅圖
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels)
plt.title('nostalgia vs. not nostalgia')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#參考:https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df['comment'])
y = df['sentiment'].apply(lambda x: 1 if x == 'nostalgia' else 0)
print(y.shape)
print(X_tfidf)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset

dataset = load_dataset("Senem/Nostalgic_Sentiment_Analysis_of_YouTube_Comments_Data")
df = pd.DataFrame(dataset['train'])

df['label'] = df['sentiment'].apply(lambda x: 1 if x == 'nostalgia' else 0)
X = df['comment']
y = df['label']


# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 詞頻
count_vectorizer = CountVectorizer()
X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

# Multinomial
nb_count = MultinomialNB()
nb_count.fit(X_train_counts, y_train)
y_pred_count = nb_count.predict(X_test_counts)
print("Multinomial NB_詞頻:")
print("Accuracy:", accuracy_score(y_test, y_pred_count))
print(classification_report(y_test, y_pred_count))

nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = nb_tfidf.predict(X_test_tfidf)
print("\nMultinomial NB_TF-IDF:")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))

# Bernoulli
nb_count1 = BernoulliNB()
nb_count1.fit(X_train_counts, y_train)
y_pred_count = nb_count1.predict(X_test_counts)
print("Bernoulli NB_詞頻:")
print("Accuracy:", accuracy_score(y_test, y_pred_count))
print(classification_report(y_test, y_pred_count))

nb_tfidf1 = BernoulliNB()
nb_tfidf1.fit(X_train_tfidf, y_train)
y_pred_tfidf = nb_tfidf1.predict(X_test_tfidf)
print("\nBernoulli NB_TF-IDF:")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf))